In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
import PyPDF2

# --- Configuration ---
PROJECT_ID = "revolut-dev-apps"
LOCATION = "us-central1"
# The specific files you provided
FILES_TO_TEST = [
    "CM-pol.pdf",
    "CMS.pdf"
]

def test_document_intelligence():
    print(f"🔄 Initializing Vertex AI in {LOCATION}...")
    vertexai.init(project=PROJECT_ID, location=LOCATION)
    model = GenerativeModel("gemini-2.5-pro")

    for file_path in FILES_TO_TEST:
        print(f"\n--- Testing File: {file_path[:50]}... ---")
        
        # 1. Physical Read Test
        text = ""
        try:
            with open(file_path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                page_count = len(reader.pages)
                # Extract first 3 pages to avoid hitting test token limits but prove readability
                for i in range(min(3, page_count)):
                    text += reader.pages[i].extract_text() + "\n"
            print(f"✅ Successfully read {page_count} pages.")
        except FileNotFoundError:
            print(f"❌ File not found. Ensure the PDF is in the same folder as this script.")
            continue
        except Exception as e:
            print(f"❌ Read Error: {e}")
            continue

        # 2. Intelligence/Summarization Test
        if text:
            print(f"🔄 Asking Gemini 2.5 Pro to summarize the core mandate...")
            prompt = f"""
            You are a Risk Auditor. Below is an excerpt from a corporate document.
            1. Summarize the 'Scope' and 'Purpose' in 3 bullet points.
            2. Identify one mandatory requirement mentioned in this text.
            
            DOCUMENT TEXT:
            {text}
            """
            try:
                response = model.generate_content(
                    prompt,
                    generation_config=GenerationConfig(temperature=0.2)
                )
                print(f"\n🤖 AI Summary of {file_path[:20]}:")
                print(f"---")
                print(response.text)
                print(f"---")
            except Exception as e:
                print(f"❌ AI Analysis Error: {e}")

if __name__ == "__main__":
    test_document_intelligence()

🔄 Initializing Vertex AI in us-central1...

--- Testing File: CM-pol.pdf... ---
✅ Successfully read 12 pages.
🔄 Asking Gemini 2.5 Pro to summarize the core mandate...


/opt/conda/lib/python3.11/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()



🤖 AI Summary of CM-pol.pdf:
---
As a Risk Auditor, here is my assessment of the provided document excerpt.

### 1. Summary of Scope and Purpose

*   **Purpose:** To establish a formal, systematic process for managing all non-technology changes, ensuring they are implemented with minimal disruption and risk to Revolut's operations and services.
*   **Scope (Applicability):** The policy applies globally to all Revolut legal entities (including subsidiaries and branches) and to all personnel (employees, contractors, and co-sourced staff).
*   **Scope (Content):** The policy governs all changes *except* for technology changes, which are covered by a separate, corresponding policy. For changes involving both, this policy governs the non-technology aspects.

### 2. Mandatory Requirement

One mandatory requirement identified in the text is:

**Any local variations or differences from this global policy must be documented within an entity-specific addendum.**

This is stated in Section 1.2.1: